<a href="https://colab.research.google.com/github/matteovergaro/Quantum_Com/blob/main/3_QL1_Qubits_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*All source material is copyright of NetSquid*

Before using this Notebook, make a copy by clicking File > Save a copy in Drive. This will save your own copy you can make changes to.

# 1.0 NetSquid Installation




In [ ]:
import os

def restart_runtime():
    os.kill(os.getpid(), 9)

# comment these 2 lines out after running
#!pip3 install --user --extra-index-url https://<username>:<password>@pypi.netsquid.org netsquid
#restart_runtime()

After running the above code block, it is recommended to comment out the following lines

```
!pip3 install --user --extra-index-url https://<username>:<password>@pypi.netsquid.org netsquid
restart_runtime()
```

In [ ]:
import netsquid as ns

# Excercise 1 - Bell State Creation
Create the Bell-State Psi Plus = (1/root(2))(|01> + |10>)

You may only use the *ns.qubits.operate()* function.

Hint: The Psi Plus state differs from the Phi Plus (1/root(2))(|00> + |11>) state by a single gate applied to the second qubit.

In [ ]:
b0, b1 = ns.qubits.create_qubits(2) # b0, b1: |0>
# ToDo:

print(ns.qubits.reduced_dm([b0, b1]))

Fidelity of your qubits vs. the Phi Plus state should be ~1.0

In [ ]:
# verify your Psi+ Bell-state
print(ns.qubits.fidelity([b0, b1], ns.b01))

# Exercise 2 - Repeated Teleportation
You will now design a similar simulation to the teleportation simulation provided in lecture, however you will repeat it twice.  

The first code block will create a Bell-pair between Alice and Bob, as well as a qubit X to be teleported from Alice to Bob. In the next code block, you will create another round of teleportation from Bob to Charlie.


Teleportation overview:
1.   Bell-Pair (alice_B, bob_B) are prepared.
2.   Alice prepares the qubit X to be teleported (alice_X) in some quantum state
3.   Alice performs a Bell-state measurement on her Bell-state (alice_B) and her qubit to be teleported (alice_X). The BSM is a joint measurement performed using the gmeasure() function.
4.   Based on the result of Alice's measurement, Bob performs certain quantum operations (i.e. corrections) to convert (i.e. teleport) his Bell-state (bob_B) into Alice_X qubit.


*Any comment with a number requires 1 single line of code below it.*

In [ ]:
ns.set_random_state(seed=40)
ns.sim_reset()

# 1. create qubit objects
# Note: to use assign_qstate(), we must set no_state=True
# alice_B, bob_B, alice_X =

# 2. Create the Bell-Pairs in state (|00> + |11>)/sqrt(2)

# verify Bell-State
print("Bell-state between Alice and Bob:\n", ns.qubits.reduced_dm([alice_B, bob_B]))

# 2. Prepare the state to be teleported in the state |1>
# Note: use ns.qubits.assign_qstate(), and ns.s1 as the state

print("alice_X state:\n", ns.qubits.reduced_dm([alice_X]))

# 3. Alice perfoms joint BSM on her Bell-state and her qubit X
# Note: use ns.qubits.gmeasure()
# meas, prob =
labels_bell = ("|00>", "|01>", "|10>", "|11>")
print(f"\nMeasured {labels_bell[meas]} with prob {prob:.2f}")
gm_alice_X = int(labels_bell[meas][1])
gm_alice_B = int(labels_bell[meas][2])


# 4. Bob may need to apply corrections to his Bell-state to recover X depending on the result of Alice's BSM
# if the outcome of the alice_B measurement is 1, apply a X-gate

# if the outcome of the alice_X measurement is 1, apply a Z-gate


# verify teleported state
print("\nVerify Bob's received teleported state\n", ns.qubits.reduced_dm([bob_B]))

Now, create a similar circuit for the teleportation of this state from Bob to Charlie.

We only need to create 2 new qubit objects this time, instead of 3 before, why is this?

We will also simulate noisy devices decreasing our teleported qubit fidelity. We will later learn how to directly simulate noise while qubits idle in quantum memory, and as they traverse quantum channels. For now, we can use the method we learned in Section 4 of the lecture material, the *delay_depolarize(depolar_rate [Hz], delay [ns])* function.

Assuming Bob creates the two entangled qubits, we need to simulate the following sources of noise and their associated delays:
- Bob's entangled qubit idling in memory before BSM (1 second / 1e9 nanoseconds)
- Bob's qubit to be teleported idling in memory before BSM (1 second / 1e9 nanoseconds).
- Charlie's entangled qubit traversing the channel from Bob to charlie (.1 second / 1e8 nanoseconds)
- Charlie's entangled qubit idling in memory while waiting for classical corrections from Bob (1 second / 1e9 nanoseconds)

Assume all quantum memories have a depolarization rate of 5e4 Hz, and quantum channels have a depolarization rate of 1e4 Hz.

You must add the *delay_depolarize()* function calls where you see fit to create this simulation (there are not numbers indicating where to put these functions).



In [ ]:
# 1. Create the necessary qubit objects
# Note: to use assign_qstate(), we must set no_state=True
# bob_B2, charlie_B =

# 2. Create the Bell-Pairs in state (|00> + |11>)/sqrt(2)

# verify Bell-State
print("Bell-state between Bob and Charlie:\n", ns.qubits.reduced_dm([bob_B2, charlie_B]))

# 3. Bob perfoms joint BSM on his Bell-state and the qubit to be teleported
# use ns.qubits.gmeasure()
# meas, prob =
labels_bell = ("|00>", "|01>", "|10>", "|11>")
print(f"\nMeasured {labels_bell[meas]} with prob {prob:.2f}")
gm_bob_B = int(labels_bell[meas][1])
gm_bob_B2 = int(labels_bell[meas][2])


# 4. Charlie may need to apply corrections to his Bell-state to recover X depending on the result of Bob's BSM
# if the outcome of the bob_B2 measurement is 1, apply a X-gate

# if the outcome of the alice_X measurement is 1, apply a Z-gate

# verify teleported state
print("\nVerify Charlies's received teleported state\n", ns.qubits.reduced_dm([charlie_B]))
# 5. verify fidelity
#fidelity =
print("Teleported state fidelity:", fidelity)

# Ex 3: Noise Investigation

We will run our first investigation with NetSquid, investigating the effect of qubit idle time on the qubit fidelity.

In Excercise 1, you will create the |B00> Bell-state then apply depolarization noise with a constant depolarization rate, over a range of delay values. In other words, we will simulate the effect of on entanglement fidelity of a constant noise rate application over variable time. We will also print the initial, pure Bell-state

Some code has been provided for you, such as the delay iteration and graph plotting. As above, any comment with a number requires you to type a one line of code below.


In [ ]:
import matplotlib.pyplot as plt

ns.set_random_state(seed=42)

# 1. create 2 qubit objects

# 2. Transform the 2 qubits into the Bell-Pairs in state (|00> + |11>)/sqrt(2) = |b00>

# 3. Print initial Bell-state density matrix

# list to hold delay values
delays = []
# list to hold fidelity of either Bell-state depolarizing
fidelities = []

# constant depolar_rate
depolar_rate = 5e4 # [Hz], Citation [1]

delay = 1
while delay < 3000:
  ns.sim_reset()
  # 3. Calculate noise probability with the current delay value and constant depolar_rate for the first qubit

  # 4. Calculate noise probability with the current delay value and constant depolar_rate for the second qubit

  # 5. calculate fidelity at delayed time
  # complete the following line...
  #fidelity =

  fidelities.append(fidelity)

  delays.append(delay)
  delay += 10

fig, ax = plt.subplots()
ax.plot(delays, fidelities)

plt.title("Fidelity vs. Time")
plt.xlabel("Time [ns]")
plt.ylabel("Bell-pair Fidelity")

# 6. Print final Bell-state density matrix
print("Initial density matrix", ns.qubits.reduced_dm([q1, q2]))

Citations
1. Choi, Joonhee, et al. "Depolarization dynamics in a strongly interacting solid-state spin ensemble." Physical review letters 118.9 (2017): 093601.